In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
import warnings
warnings.filterwarnings("ignore")

# 1. Data Preparation

In [7]:
## input training set ## 90960 x 20 df
df_ROI_train = pd.read_csv(r"H:\3_output_raMSIn\3_3_Output_raMSI_HKU_Ingested4FNA\0_dbMSI_10ppm\postPredict10ppm_tPRfNRfDR_train.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [8]:
## input ext val set ## 6075 x 20 df
df_ROI_ext = pd.read_csv(r"H:\3_output_raMSIn\3_3_Output_raMSI_HKU_Ingested4FNA\0_dbMSI_10ppm\postPredict10ppm_tPRfNRfDR_ext.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [9]:
## reconstruct a whole set ## 97035 x 20 df
df_ROI_ingested = pd.concat([df_ROI_train, df_ROI_ext]).reset_index().drop(columns=["index"])

In [10]:
## input FNA set ## 88701 x 20 df
df_ROI_FNA = pd.read_csv(r"H:\3_output_raMSIn\3_3_Output_raMSI_HKU_Ingested4FNA\0_dbMSI_10ppm\postPredict10ppm_tPRfNRfDR_FNA.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [11]:
df_ROI_ingested

,pixel_id,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,609.5114,682.5922,846.5858,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.312924,-0.028178,0.243141,0.063224,0.318632,0.176635,0.120440,-0.050148,-0.229186,...,0,1,0.0,1.0,FP,0.987245,0.012755,0.060907,0.058716,0.941284
1,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.286665,0.004161,0.344826,0.045231,0.358340,0.117522,0.129577,0.032616,-0.229186,...,0,1,0.0,1.0,FP,0.987245,0.012755,0.060907,0.058716,0.941284
2,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.354700,0.010371,0.131824,0.098479,0.250928,0.304289,0.138565,0.070119,0.200686,...,0,1,0.0,1.0,FP,0.987245,0.012755,0.060907,0.058716,0.941284
3,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.239929,0.064175,0.295961,-0.008660,0.283534,0.181531,0.232746,-0.033225,-0.229186,...,0,1,0.0,1.0,FP,0.987245,0.012755,0.060907,0.058716,0.941284
4,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.406599,0.026178,-0.239962,-0.009675,0.212457,0.063778,0.127102,-0.035991,0.172265,...,0,1,0.0,1.0,FP,0.987245,0.012755,0.060907,0.058716,0.941284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97030,HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4...,-0.223985,-0.042569,-0.139794,0.036082,-0.103957,-0.130373,-0.087227,-0.230339,-0.012480,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.484444,1.000000,0.000000
97031,HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4...,-0.223985,-0.004552,-0.085881,0.014525,-0.150224,-0.130406,-0.093047,-0.230339,-0.012480,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.484444,1.000000,0.000000
97032,HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4...,-0.223985,-0.024077,-0.120811,0.026870,-0.117257,-0.118881,-0.072588,-0.230339,-0.012480,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.484444,1.000000,0.000000
97033,HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4...,-0.223985,-0.123345,-0.130016,0.012985,-0.116073,-0.195635,-0.084656,0.125360,-0.012480,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.484444,1.000000,0.000000


In [12]:
df_ROI_train["Sample"] = 0
df_ROI_train["Row"] = 0
df_ROI_train["Scan"] = 0
df_ROI_train_Sample = []
df_ROI_train_Row = []
df_ROI_train_Scan = []
for i in range(len(df_ROI_train["pixel_id"])):
    df_ROI_train_Sample.append("_".join(str(df_ROI_train["pixel_id"][i]).split("_")[:-2]))
    df_ROI_train_Row.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-1]))
    df_ROI_train_Scan.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-2]))
df_ROI_train["Sample"] = df_ROI_train_Sample
df_ROI_train["Row"] = df_ROI_train_Row
df_ROI_train["Scan"] = df_ROI_train_Scan
pop_column = df_ROI_train.pop('Scan')
df_ROI_train.insert(1, 'Scan', pop_column)
pop_column = df_ROI_train.pop('Row')
df_ROI_train.insert(1, 'Row', pop_column)
pop_column = df_ROI_train.pop('Sample')
df_ROI_train.insert(1, 'Sample', pop_column)

df_ROI_ext["Sample"] = 0
df_ROI_ext["Row"] = 0
df_ROI_ext["Scan"] = 0
df_ROI_ext_Sample = []
df_ROI_ext_Row = []
df_ROI_ext_Scan = []
for i in range(len(df_ROI_ext["pixel_id"])):
    df_ROI_ext_Sample.append("_".join(str(df_ROI_ext["pixel_id"][i]).split("_")[:-2]))
    df_ROI_ext_Row.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-1]))
    df_ROI_ext_Scan.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-2]))
df_ROI_ext["Sample"] = df_ROI_ext_Sample
df_ROI_ext["Row"] = df_ROI_ext_Row
df_ROI_ext["Scan"] = df_ROI_ext_Scan
pop_column = df_ROI_ext.pop('Scan')
df_ROI_ext.insert(1, 'Scan', pop_column)
pop_column = df_ROI_ext.pop('Row')
df_ROI_ext.insert(1, 'Row', pop_column)
pop_column = df_ROI_ext.pop('Sample')
df_ROI_ext.insert(1, 'Sample', pop_column)

df_ROI_ingested["Sample"] = 0
df_ROI_ingested["Row"] = 0
df_ROI_ingested["Scan"] = 0
df_ROI_ingested_Sample = []
df_ROI_ingested_Row = []
df_ROI_ingested_Scan = []
for i in range(len(df_ROI_ingested["pixel_id"])):
    df_ROI_ingested_Sample.append("_".join(str(df_ROI_ingested["pixel_id"][i]).split("_")[:-2]))
    df_ROI_ingested_Row.append(int(str(df_ROI_ingested["pixel_id"][i]).split("_")[-1]))
    df_ROI_ingested_Scan.append(int(str(df_ROI_ingested["pixel_id"][i]).split("_")[-2]))
df_ROI_ingested["Sample"] = df_ROI_ingested_Sample
df_ROI_ingested["Row"] = df_ROI_ingested_Row
df_ROI_ingested["Scan"] = df_ROI_ingested_Scan
pop_column = df_ROI_ingested.pop('Scan')
df_ROI_ingested.insert(1, 'Scan', pop_column)
pop_column = df_ROI_ingested.pop('Row')
df_ROI_ingested.insert(1, 'Row', pop_column)
pop_column = df_ROI_ingested.pop('Sample')
df_ROI_ingested.insert(1, 'Sample', pop_column)

df_ROI_FNA["Sample"] = 0
df_ROI_FNA["Row"] = 0
df_ROI_FNA["Scan"] = 0
df_ROI_FNA_Sample = []
df_ROI_FNA_Row = []
df_ROI_FNA_Scan = []
for i in range(len(df_ROI_FNA["pixel_id"])):
    df_ROI_FNA_Sample.append("_".join(str(df_ROI_FNA["pixel_id"][i]).split("_")[:-2]))
    df_ROI_FNA_Row.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-1]))
    df_ROI_FNA_Scan.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-2]))
df_ROI_FNA["Sample"] = df_ROI_FNA_Sample
df_ROI_FNA["Row"] = df_ROI_FNA_Row
df_ROI_FNA["Scan"] = df_ROI_FNA_Scan
pop_column = df_ROI_FNA.pop('Scan')
df_ROI_FNA.insert(1, 'Scan', pop_column)
pop_column = df_ROI_FNA.pop('Row')
df_ROI_FNA.insert(1, 'Row', pop_column)
pop_column = df_ROI_FNA.pop('Sample')
df_ROI_FNA.insert(1, 'Sample', pop_column)

In [13]:
df_ROI_train = df_ROI_train.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_ext = df_ROI_ext.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_ingested = df_ROI_ingested.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_FNA = df_ROI_FNA.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [14]:
df_ROI_FNA

,pixel_id,Sample,Row,Scan,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,4,-0.004859,0.014806,-0.163063,0.120670,-0.106771,-0.055432,...,0,1,0.0,1.0,FP,0.876112,0.123888,0.398794,0.576201,0.423799
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,104,140,-0.004859,-0.287681,0.145310,-0.143080,0.589844,-0.055432,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.502136,1.000000,0.000000
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,104,139,-0.004859,0.155660,0.143503,0.173083,0.462333,-0.055432,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.502136,1.000000,0.000000
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,104,138,-0.004859,0.138036,0.153061,-0.143080,0.628322,-0.055432,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.502136,1.000000,0.000000
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,104,134,-0.004859,0.090796,0.141841,-0.143080,0.443829,-0.055432,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.502136,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88696,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2,50,112,-0.004859,-0.287681,-0.163063,-0.143080,-0.106771,-0.055432,...,1,1,0.0,1.0,TP,0.876112,0.123888,0.398794,0.576201,0.423799
88697,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2,50,113,-0.004859,-0.287681,-0.163063,-0.143080,-0.106771,-0.055432,...,1,1,0.0,1.0,TP,0.876112,0.123888,0.398794,0.576201,0.423799
88698,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2,50,115,-0.004859,-0.287681,0.074818,-0.143080,-0.106771,-0.055432,...,1,1,0.0,1.0,TP,0.876112,0.123888,0.398794,0.576201,0.423799
88699,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2,50,117,-0.004859,0.019156,-0.163063,-0.143080,-0.106771,-0.055432,...,1,1,0.0,1.0,TP,0.876112,0.123888,0.398794,0.576201,0.423799


In [15]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [16]:
sample_ROI_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI]

In [17]:
name_list = ["HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI"]

In [18]:
sample_ROI_list_train = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI]

sample_ROI_list_ext = [HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_ingested = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_FNA = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI]

In [19]:
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI

,pixel_id,Sample,Row,Scan,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,4,-0.004859,0.014806,-0.163063,0.120670,-0.106771,-0.055432,...,0,1,0.0,1.0,FP,0.876112,0.123888,0.398794,0.576201,0.423799
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,5,-0.004859,-0.287681,0.069112,0.423220,-0.106771,-0.055432,...,0,1,0.0,1.0,FP,0.876112,0.123888,0.398794,0.576201,0.423799
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,6,-0.004859,-0.287681,0.101541,0.375696,-0.106771,-0.055432,...,0,1,0.0,1.0,FP,0.876112,0.123888,0.398794,0.576201,0.423799
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,194,-0.004859,-0.287681,0.663453,-0.143080,-0.106771,-0.055432,...,0,1,0.0,1.0,FP,0.876112,0.123888,0.398794,0.576201,0.423799
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,195,-0.004859,-0.287681,0.603402,-0.143080,-0.106771,-0.055432,...,0,1,0.0,1.0,FP,0.876112,0.123888,0.398794,0.576201,0.423799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,144,-0.004859,0.052979,0.114157,-0.143080,-0.106771,-0.055432,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.502136,1.000000,0.000000
14821,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,145,-0.004859,-0.287681,0.110845,-0.143080,-0.106771,-0.055432,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.502136,1.000000,0.000000
14822,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,146,-0.004859,-0.287681,0.097728,-0.143080,-0.106771,-0.055432,...,0,1,0.0,1.0,FP,0.876112,0.123888,0.398794,0.576201,0.423799
14823,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,118,177,-0.004859,-0.287681,0.482772,0.201250,-0.106771,-0.055432,...,0,1,0.0,1.0,FP,0.876112,0.123888,0.398794,0.576201,0.423799


In [20]:
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI

,pixel_id,Sample,Row,Scan,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_5_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,5,-0.004859,0.177981,0.209629,-0.143080,-0.106771,-0.055432,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.502136,1.000000,0.000000
1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_6_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,6,-0.004859,0.418624,0.278727,0.311384,-0.106771,-0.055432,...,1,1,0.0,1.0,TP,0.876112,0.123888,0.398794,0.576201,0.423799
2,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_7_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,7,-0.004859,0.192563,0.293220,0.367746,-0.106771,-0.055432,...,1,1,0.0,1.0,TP,0.876112,0.123888,0.398794,0.576201,0.423799
3,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_8_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,8,-0.004859,0.242601,0.364789,0.369629,-0.106771,-0.055432,...,1,1,0.0,1.0,TP,0.876112,0.123888,0.398794,0.576201,0.423799
4,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_9_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,9,-0.004859,-0.287681,0.313500,0.189476,-0.106771,-0.055432,...,1,1,0.0,1.0,TP,0.876112,0.123888,0.398794,0.576201,0.423799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14249,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,189,-0.004859,0.143177,0.374268,0.250686,-0.106771,-0.055432,...,1,1,0.0,1.0,TP,0.876112,0.123888,0.398794,0.576201,0.423799
14250,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,190,-0.004859,0.092616,0.344487,0.279352,-0.106771,-0.055432,...,1,1,0.0,1.0,TP,0.876112,0.123888,0.398794,0.576201,0.423799
14251,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,191,-0.004859,0.087575,0.359319,0.197591,-0.106771,-0.055432,...,1,1,0.0,1.0,TP,0.876112,0.123888,0.398794,0.576201,0.423799
14252,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,126,185,-0.004859,0.195149,0.388687,0.304119,-0.106771,-0.055432,...,1,1,0.0,1.0,TP,0.876112,0.123888,0.398794,0.576201,0.423799


# 2. Prediction (made)

## deploy models for training set ##
model_xgb = jl.load(r"H:\3_output_raMSIn\modelXGBoost.joblib")
#
## deploy model ##
predicted0n1_train = model_xgb.predict(X_train)
trainDEFSDf["predicted0n1"] = predicted0n1_train
#
## save ##, ouputing trainSet df
savePath = r"H:\3_output_raMSIn\df_train24_std_0n1.csv"
trainDEFSDf.to_csv(savePath)

pTP_train = model_xgb.predict_proba(X_train)
trainDEFSDf["p(0)"] = list(pd.DataFrame(pTP_train)[0])
trainDEFSDf["p(1)"] = list(pd.DataFrame(pTP_train)[1])
#
## save ##, ouputing trainSet df
savePath = r"H:\3_output_raMSIn\df_train24_std_0n1_pTP.csv"
trainDEFSDf.to_csv(savePath)

In [21]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["p(1)"]).count(1)) ) *100, "%"

(64.75548060708263, '%')

In [22]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["p(1)"]).count(1)) ) *100, "%"

(58.62545806540829, '%')

In [23]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["p(1)"]).count(1)) ) *100, "%"

(49.73118279569893, '%')

In [24]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["p(1)"]).count(1)) ) *100, "%"

(98.28819980356391, '%')

In [25]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["p(1)"]).count(1)) ) *100, "%"

(90.68508363989727, '%')

In [26]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["p(1)"]).count(1)) ) *100, "%"

(74.93548387096774, '%')

# 3. EDA

In [27]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [28]:
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm
import numpy as np
from matplotlib.colors import ListedColormap
cmap = cm.binary

def rgb_white2alpha(rgb, ensure_increasing=False):

    # The most transparent alpha we can use is given by the min of RGB
    # Convert it from saturation to opacity
    alpha = 1. - np.min(rgb, axis=1)
    if ensure_increasing:
        # Let's also ensure the alpha value is monotonically increasing
        a_max = alpha[0]
        for i, a in enumerate(alpha):
            alpha[i] = a_max = np.maximum(a, a_max)
    alpha = np.expand_dims(alpha, -1)
    # Rescale colors to discount the white that will show through from transparency
    rgb = (rgb + alpha - 1) / alpha
    # Concatenate our alpha channel
    return np.concatenate((rgb, alpha), axis=1)
    

def cmap_white2alpha(name, ensure_increasing=False, register=True):

    # Fetch the cmap callable
    cmap = plt.get_cmap(name)
    # Get the colors out from the colormap LUT
    rgb = cmap(np.arange(cmap.N))[:, :3]  # N-by-3
    # Convert white to alpha
    rgba = rgb_white2alpha(rgb, ensure_increasing=ensure_increasing)
    # Create a new Colormap object
    cmap_alpha = matplotlib.colors.ListedColormap(rgba, name=name + "_alpha")
    if register:
        matplotlib.cm.register_cmap(name=name + "_alpha", cmap=cmap_alpha)
    return cmap_alpha

In [40]:
# Get our new version of the colormap with white2alpha
cmap_w2a = cmap_white2alpha("Reds")

In [41]:
def genAROItif(sample_ROI, sample_, name, mz, vmax_=1):

    df_heatmap = sample_ROI[["Row", "Scan", mz]]
    df_heatmap2 = pd.DataFrame(np.array([[0, 0, 0]]), columns=['Row', 'Scan', mz]).reset_index().drop(columns = ["index"])
    df_HM = df_heatmap2.loc[df_heatmap["Row"] == 0].loc[df_heatmap2["Scan"] == 0]
    Heat_Row = sample_["Row"]
    Heat_Scan = sample_["Scan"]


    #hi = 0
    for y in range(len(set(Heat_Row))):
        for x in range(1, len(set(Heat_Scan))+1):

            df_in = df_heatmap.loc[df_heatmap["Row"] == (y+1)].loc[df_heatmap["Scan"] == x]
            if len(df_in) == 1:
                df_HM = pd.concat([df_HM, df_in], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


            elif len(df_in) == 0:
                df_heatmap2["Row"] = y+1
                df_heatmap2["Scan"] = x
                df_HM = pd.concat([df_HM, df_heatmap2], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


    #print(name, mz, hi, len(df_HM))
    if "index" in df_HM.columns:
        df_HM = df_HM.drop(columns = ["index", "Row", "Scan"])
    else:
        df_HM = df_HM.drop(columns = ["Row", "Scan"])

    df_HM = df_HM.T
    New_y = []
    for y in range(1, len(set(Heat_Row))+1):
        New_y.append(str(y))
    New_x = []
    for x in range(1, len(set(Heat_Scan))+1):
        New_x.append(str(x))

    arr = df_HM.values.copy()
    arr.resize(len(set(Heat_Row)), len(set(Heat_Scan)))

    df_HM = pd.DataFrame(arr)
    df_HM.index = New_y
    df_HM.columns = New_x

    for i in list(df_HM.columns):
        df_HM[i] = df_HM[i].astype(float)

    plt.subplots(dpi = 300)
    plt.imshow(df_HM, cmap=cmap_w2a, vmin=0, vmax=1, zorder=1) #cmap_w2a
    plt.xticks([])
    plt.yticks([])
    plt.box(False)
    plt.autoscale(enable=True, tight=True)

    plt.rcParams['figure.figsize'] = [2, 2]

    plt.savefig(f"H:/3_output_raMSIn/3_3_Output_raMSI_HKU_Ingested4FNA/0_dbMSI_10ppm/{name}_{mz}.tif", bbox_inches = 'tight', pad_inches = 0)

In [31]:
df_raMSI_ML = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSI_HKU_Ingested4FNA/0_dbMSI_10ppm/dbMSI10ppm_noLT.csv').set_index('pixel_id')
df_raMSI_ML.shape

(371480, 2603)

In [32]:
raMSI_ML = df_raMSI_ML.reset_index()

In [33]:
raMSI_ML["Sample"] = 0
raMSI_ML["Row"] = 0
raMSI_ML["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(raMSI_ML["pixel_id"])):
    Sample.append("_".join(str(raMSI_ML["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-2]))
raMSI_ML["Sample"] = Sample
raMSI_ML["Row"] = Row
raMSI_ML["Scan"] = Scan

In [34]:
pop_column = raMSI_ML.pop('Scan')
raMSI_ML.insert(1, 'Scan', pop_column)
pop_column = raMSI_ML.pop('Row')
raMSI_ML.insert(1, 'Row', pop_column)
pop_column = raMSI_ML.pop('Sample')
raMSI_ML.insert(1, 'Sample', pop_column)

In [35]:
raMSI_ML = raMSI_ML.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [36]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [37]:
sample_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6, 
               HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2]

In [42]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

TOP ="p(1)"
    

counter = 0

for im in range(len(sample_ROI_list)):
    start = datetime.now()

    name = name_list[im]
    sample_ = sample_list[im]
    preview = sample_ROI_list[im][[TOP]]
    for i in preview.columns:
        preview[i] = preview[i].astype(float)

    vmax_list = list(preview.describe().iloc[-1])
    vmax_ = 1
    #print(TOP, name, vmax_)

    genAROItif(sample_ROI=sample_ROI_list[im], sample_=sample_, name=name, mz=TOP, vmax_=vmax_)
    #print(df.shape)
    counter += 1
    end = datetime.now()
    Duration = end - start
    print(Duration)


0:01:09.113972
0:01:28.229262
0:02:04.584767
0:01:05.511597
0:01:18.214693
0:01:46.480771
0:00:06.502557
0:00:04.773671
0:00:07.415740
0:00:07.043556
0:03:58.605858
0:03:54.143689
0:04:10.997830
0:01:53.525182
0:01:32.220779
0:01:17.812436
0:01:16.885225
0:01:15.102415
0:01:16.742748
0:00:04.614559
0:00:11.210527
0:00:06.425741
0:00:05.430840
0:03:42.904612
0:03:49.921986
0:03:21.134278
